<a href="https://colab.research.google.com/github/Prashanth2902/CS6120-Assignment-2/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [22]:
import json
import requests
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [ ]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [23]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  documents = []
  genres = []

  for doc in corpus:
    genres.append(doc['genre'])
    tokens = doc['text'].split()
    alpha_tokens = [token.lower() for token in tokens if re.search('[a-zA-Z]', token)]
    documents.append(" ".join(alpha_tokens))

  vectorizer = CountVectorizer()
  texts = vectorizer.fit_transform(documents)
  vocab = vectorizer.get_feature_names_out().tolist()
  return texts,genres,vocab

In [24]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  tragedy_words = [
      'blood', 'death', 'die', 'kill', 'sorrow', 'sad', 'curse', 'hell',
      'grief', 'fear', 'king', 'queen', 'prince', 'lord', 'war', 'fate',
      'god', 'heaven', 'revenge', 'poison', 'dead', 'ghost'
  ]

  comedy_words = [
      'love', 'marry', 'sweet', 'happy', 'joy', 'laugh', 'fool', 'jest',
      'wit', 'sir', 'lady', 'mistress', 'sing', 'dance', 'wedding', 'woo',
      'kiss', 'wife', 'husband', 'merry'
  ]
  interest_vocab = tragedy_words + comedy_words

  genres = [play['genre'] for play in corpus]
  documents = [play['text'] for play in corpus]

  vectorizer = CountVectorizer(vocabulary=interest_vocab, lowercase=True)

  texts = vectorizer.fit_transform(documents)
  vocab = vectorizer.get_feature_names_out().tolist()

  return texts,genres,vocab

In [25]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [26]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [27]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'your', 'you', 'for', 'me', 'duke', 'of', 'love', 'presence', 'prettier', 'pretia', 'pretext', 'pretense', 'pretending', 'pretended', 'pretend', 'pretences', 'present']
The most informative terms for neg are: ['our', 'him', 'rom', 'iago', 'thy', 'ham', 'imogen', 'what', 'his', 'brutus', 'lear', 'timon', 'present', 'preserv', 'prettier', 'pretia', 'pretext', 'pretense', 'pretending', 'pretended']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['you', 'prospero', 'duke', 'helena', 'antonio', 'me', 'for', 'your', 'sir', 'ariel', 'sebastian', 'hermia', 'lysander', 'parolles', 'stephano', 'will', 'leontes', 'caliban', 'demetrius', 'love']
The most informative terms for neg are: ['ham', 'iago', 'him', 'our', 'othello', 'what', 'his', 'lear', 'imogen', 'brutus', 'rom', 'nurse', 'romeo', 'caesa

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [28]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.692308
The most informative terms for pos are: ['sir', 'love', 'prince', 'king', 'sweet', 'curse', 'kill', 'die', 'fate', 'lord', 'fear', 'grief', 'god', 'hell', 'sad', 'sorrow', 'laugh', 'jest', 'joy', 'happy']
The most informative terms for neg are: ['war', 'dead', 'fool', 'death', 'heaven', 'blood', 'queen', 'wife', 'lady', 'curse', 'hell', 'grief', 'fate', 'sad', 'fear', 'sorrow', 'wit', 'happy', 'lord', 'revenge']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['prince', 'sir', 'merry', 'sad', 'love', 'mistress', 'marry', 'sweet', 'king', 'sing', 'dance', 'husband', 'fear', 'joy', 'jest', 'die', 'happy', 'woo', 'wedding', 'laugh']
The most informative terms for neg are: ['war', 'death', 'fool', 'blood', 'dead', 'kill', 'poison', 'wife', 'queen', 'grief', 'lady', 'ghost', 'heaven', 'revenge', 'hell', 'wit', 'curse', 'kiss', 'fate', 'lord']


**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?